# TODO

In [1]:
from qiskit import *
from qiskit.visualization import plot_histogram

from random import getrandbits, randint

from binary_cost_function import *
from custom_gas import *
from num_base_converter import *

In [1]:
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
from qiskit import BasicAer
from docplex.mp.model import Model
from qiskit import IBMQ

import numpy as np

import time
import stopit

import pickle

import math

Loading both simulator and real quantum machine backends.

In [2]:
#Quantum machine simulator backend
sim_backend = BasicAer.get_backend('statevector_simulator')

"""
#Read credentials from file
creds = {}
with open('creds.txt') as f:
    for line in f:
        (key, val) = line.split()
        creds[key] = val

#Sign in with account credentials
IBMQ.save_account(creds['API_KEY'], overwrite=True)
IBMQ.load_account()
provider = IBMQ.get_provider(hub = 'ibm-q')

#Real quantum machine backends
real_16q_backend = provider.get_backend('ibmq_16_melbourne')
real_5q_backend = provider.get_backend('ibmq_santiago')
"""

"\n#Read credentials from file\ncreds = {}\nwith open('creds.txt') as f:\n    for line in f:\n        (key, val) = line.split()\n        creds[key] = val\n\n#Sign in with account credentials\nIBMQ.save_account(creds['API_KEY'], overwrite=True)\nIBMQ.load_account()\nprovider = IBMQ.get_provider(hub = 'ibm-q')\n\n#Real quantum machine backends\nreal_16q_backend = provider.get_backend('ibmq_16_melbourne')\nreal_5q_backend = provider.get_backend('ibmq_santiago')\n"

Creating QUBO to optimize. Chosen test QUBO will be:

$$\mathcal{C}(x) = x^{\top} \cdot \begin{pmatrix}0&6&-3\\0&0&-1\\0&0&0\end{pmatrix} \cdot x + \begin{pmatrix}2\\1\\-1\end{pmatrix} \cdot x.$$

The name of the params will be:

$$
A = \begin{pmatrix}0&6&-3\\0&0&-1\\0&0&0\end{pmatrix};\:\: b^{\top} = \begin{pmatrix}2\\1\\-1\end{pmatrix} 
$$

In [3]:
A = np.array([[0.0, 6.0, -3.0],[0.0, 0.0, -1.0],[0.0, 0.0, 0.0]])
b = np.array([2.0, 1.0, -1.0])

In [4]:
def generate_qubo(A, b, verbose = False):
    num_variables = A.shape[0]

    qubo = QuadraticProgram()
    for i in range(num_variables):
        qubo.binary_var('x' + str(i))
    qubo.minimize(linear = b, quadratic = A)
    
    if verbose:
        print(qubo.export_as_lp_string())
    
    return qubo

qubo = generate_qubo(A, b, True)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 2 x0 + x1 - x2 + [ 12 x0*x1 - 6 x0*x2 - 2 x1*x2 ]/2
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1

Binaries
 x0 x1 x2
End



Use brute force to check te correct result.

In [5]:
def next_binary(x, l):
    x[0] += 1
    for i in range(l-1):
        if x[i] == 2:
            x[i] = 0
            x[i+1] += 1
    x[-1] %= 2
    return x

def eval_cost(A, b, x):
    arr_x = np.array(x)
    return np.matmul(np.matmul(arr_x.T, A), arr_x) + np.matmul(b.T, arr_x)

def solve_brute_force_qubo(A, b, verbose = False):
    num_variables = b.size
    x = [1] * num_variables
    best_x = None
    best_fval = float('inf')
    for i in range(2 ** num_variables):
        x = next_binary(x, num_variables)
        fval = eval_cost(A, b, x)
        if best_fval > fval:
            best_fval = fval
            best_x = x.copy()
        if verbose:
            print("x =", x, "---> C(x) =", eval_cost(A, b, x))
    if verbose:
        print("   --- BEST STRING ---")
        print("x =", best_x, "---> C(x) =", best_fval)
    return {
        "best_x": best_x,
        "best_fval": best_fval
    }

solve_brute_force_qubo(A, b, verbose = True)

x = [0, 0, 0] ---> C(x) = 0.0
x = [1, 0, 0] ---> C(x) = 2.0
x = [0, 1, 0] ---> C(x) = 1.0
x = [1, 1, 0] ---> C(x) = 9.0
x = [0, 0, 1] ---> C(x) = -1.0
x = [1, 0, 1] ---> C(x) = -2.0
x = [0, 1, 1] ---> C(x) = -1.0
x = [1, 1, 1] ---> C(x) = 4.0
   --- BEST STRING ---
x = [1, 0, 1] ---> C(x) = -2.0


{'best_x': [1, 0, 1], 'best_fval': -2.0}

Apply the ```Grover Optimizer``` using each of the backends with different parameters to test it.

In [6]:
@stopit.threading_timeoutable(default = None)
def run_grover(grover_optimizer, qubo):
    return grover_optimizer.solve(qubo)

def test_grover(qubo, num_qubits, num_iterations, backend, correct_x, correct_fval, timeout_secs):
    params = {
        "num_qubits": num_qubits,
        "num_iterations": num_iterations,
    }
    
    try:
        grover_optimizer = GroverOptimizer(
            num_qubits,
            num_iterations = num_iterations,
            quantum_instance = backend)
        start_time = time.time()
        results = run_grover(grover_optimizer, qubo, timeout = timeout_secs)
        end_time = time.time()
    except Exception as ex:
        return {
            "status": "Error",
            "params": params,
            "error_msg": type(ex).__name__
        }
        
    if results:
        return {
            "status": "Success" if ((results.x == correct_x).all() and results.fval == correct_fval) else "Failure",
            "params": params,
            "results": {
                "x": results.x,
                "fval": results.fval
            }, 
            "time_secs": end_time - start_time
        }
    else:
        return {
            "status": "Timeout",
            "params": params
        }

def perform_grover_tests(nums_qubits, nums_iterations, repetitions, backend, A, b, timeout_secs, verbose = False):
    brute_force_results = solve_brute_force_qubo(A, b)
    qubo = generate_qubo(A, b)
    test_result_list = []
    for num_qubits in nums_qubits:
        for num_iterations in nums_iterations:
            if verbose:
                print("Testing {0:2d} qubits, {1:2d} iterations".format(num_qubits, num_iterations))
            for repetition_num in [i+1 for i in range(repetitions)]:
                if verbose:
                    print("  Repetition {0:3d}/{1:3d}... ".format(repetition_num, repetitions), end = "")
                test_result = test_grover(qubo, num_qubits, num_iterations, backend, brute_force_results['best_x'], brute_force_results['best_fval'], timeout_secs)
                test_result_list.append(test_result)
                if verbose:
                    print(test_result["status"])
    return test_result_list

In [7]:
start_time = time.time()

nums_qubits = [3, 4, 5, 6]
nums_iterations = [1, 2, 3, 4]
repetitions = 5
test_result_list = perform_grover_tests(nums_qubits, nums_iterations, repetitions, sim_backend, A, b, 60, verbose = True)

end_time = time.time()
secs = end_time - start_time
hours = int(math.floor(secs / 3600))
mins = int(math.floor(secs / 60))
mills = int(1000 * round(secs - math.floor(secs), 3))
secs = math.floor(secs % 60)
print("Total tests: {0:d}".format(len(nums_qubits) * len(nums_iterations) * repetitions))
print("Total time elapsed: {0:02d}:{1:02d}:{2:02d}.{3:03d}".format(hours, mins, secs, mills))

Testing  3 qubits,  1 iterations
  Repetition   1/  5... Error
  Repetition   2/  5... Success
  Repetition   3/  5... Failure
  Repetition   4/  5... Success
  Repetition   5/  5... Failure
Testing  3 qubits,  2 iterations
  Repetition   1/  5... Error
  Repetition   2/  5... Failure
  Repetition   3/  5... Error
  Repetition   4/  5... Error
  Repetition   5/  5... Error
Testing  3 qubits,  3 iterations
  Repetition   1/  5... Failure
  Repetition   2/  5... Failure
  Repetition   3/  5... Error
  Repetition   4/  5... Error
  Repetition   5/  5... Error
Testing  3 qubits,  4 iterations
  Repetition   1/  5... Error
  Repetition   2/  5... Failure
  Repetition   3/  5... Failure
  Repetition   4/  5... Error
  Repetition   5/  5... Error
Testing  4 qubits,  1 iterations
  Repetition   1/  5... Success
  Repetition   2/  5... Success
  Repetition   3/  5... Failure
  Repetition   4/  5... Success
  Repetition   5/  5... Failure
Testing  4 qubits,  2 iterations
  Repetition   1/  5... 

Save or read results into a text file.

In [8]:
def write_results(results, filename):
    f = open(filename, "wb")
    pickle.dump(results, f)
    f.close()
    
def read_results(filename):
    f = open(filename, "rb")
    results = pickle.load(f)
    f.close()
    return results

In [9]:
write_results(test_result_list, "results1.pkl")